In [107]:
import requests
import json

In [108]:
# AC_URL = "https://answercoalesce.renci.org/query"
AC_URL =  "https://answer-coalesce.ci.transltr.io/query"

In [109]:
def generate_trapi_query(input_node_type,output_node_type,input_curies,predicate,input_is_subject=True):
    envelope = {"message":{"query_graph": {"nodes":{"input":{},"output":{}}, "edges":{"edge_0":{}}}}}
    input_node = envelope["message"]["query_graph"]["nodes"]["input"]
    input_node["categories"] = [input_node_type]
    input_node["ids"] = ["uuid:1"]
    input_node["member_ids"] = input_curies
    input_node["set_interpretation"]="MANY"
    output_node = envelope["message"]["query_graph"]["nodes"]["output"]
    output_node["categories"] = [output_node_type]
    query_edge = envelope["message"]["query_graph"]["edges"]["edge_0"]
    if input_is_subject:
        query_edge["subject"] = "input"
        query_edge["object"] = "output"
    else:
        query_edge["subject"] = "output"
        query_edge["object"] = "input"
    query_edge["predicates"]  = [ predicate ]
    #query_edge["knowledge_type"] = "inferred"
    return envelope

In [110]:
#Phenotype to Disease
input_type = "biolink:PhenotypicFeature"
output_type = "biolink:Disease"
predicate = "biolink:has_phenotype"
input_is_subject = "False"

In [111]:
#Leigh Disease?
leigh_phenotypes = ["HP:0000739", #| Anxiety, 
                    "HP:0001288", # | Gait disturbance, 
                    "HP:0001252", # | Hypotonia, 
                    "HP:0001250", # | Seizure, 
                    "HP:0000750", # | Delayed speech and language development, 
                    "HP:0002378", # | Hand tremor, 
                    "HP:0002019", # | Constipation, 
                    "HP:0007146"] # | Bilateral basal ganglia lesions

In [112]:
leigh_query = generate_trapi_query( input_type, output_type, leigh_phenotypes, predicate, input_is_subject)
response = requests.post( AC_URL, json = leigh_query )
print( response.status_code )

200


In [113]:
response.json()

{'message': {'query_graph': {'nodes': {'input': {'ids': ['uuid:1'],
     'categories': ['biolink:PhenotypicFeature'],
     'set_interpretation': 'MANY',
     'constraints': [],
     'member_ids': ['HP:0000739',
      'HP:0001288',
      'HP:0001252',
      'HP:0001250',
      'HP:0000750',
      'HP:0002378',
      'HP:0002019',
      'HP:0007146']},
    'output': {'categories': ['biolink:Disease'],
     'set_interpretation': 'BATCH',
     'constraints': []}},
   'edges': {'edge_0': {'subject': 'input',
     'object': 'output',
     'predicates': ['biolink:has_phenotype'],
     'attribute_constraints': [],
     'qualifier_constraints': []}}},
  'knowledge_graph': {'nodes': {'HP:0000739': {'categories': ['biolink:PhenotypicFeature'],
     'attributes': []},
    'HP:0001288': {'categories': ['biolink:PhenotypicFeature'],
     'attributes': []},
    'HP:0001252': {'categories': ['biolink:PhenotypicFeature'],
     'attributes': []},
    'HP:0001250': {'categories': ['biolink:PhenotypicFeat

In [114]:
leigh_query

{'message': {'query_graph': {'nodes': {'input': {'categories': ['biolink:PhenotypicFeature'],
     'ids': ['uuid:1'],
     'member_ids': ['HP:0000739',
      'HP:0001288',
      'HP:0001252',
      'HP:0001250',
      'HP:0000750',
      'HP:0002378',
      'HP:0002019',
      'HP:0007146'],
     'set_interpretation': 'MANY'},
    'output': {'categories': ['biolink:Disease']}},
   'edges': {'edge_0': {'subject': 'input',
     'object': 'output',
     'predicates': ['biolink:has_phenotype']}}}}}

In [115]:
resp = response.json()
resp["message"].keys()

dict_keys(['query_graph', 'knowledge_graph', 'results', 'auxiliary_graphs'])

In [116]:

for result in resp["message"]["results"]:
    nb = result["node_bindings"]["output"][0]["id"]
    name = resp["message"]["knowledge_graph"]["nodes"][nb]["name"]
    print(nb, name)
    

MONDO:0859085 neurodevelopmental disorder with gait disturbance, dysmorphic facies, and behavioral abnormalities, X-linked
MONDO:0014553 Tenorio syndrome
MONDO:0060761 neurodevelopmental disorder with epilepsy and hypoplasia of the corpus callosum
MONDO:0013761 childhood encephalopathy due to thiamine pyrophosphokinase deficiency
MONDO:0010496 X-linked intellectual disability-short stature-overweight syndrome
MONDO:0012342 7q11.23 microduplication syndrome
MONDO:0010305 creatine transporter deficiency
MONDO:0008939 isolated cerebellar hypoplasia/agenesis
MONDO:0012423 MORM syndrome
MONDO:0014953 gnb5-related intellectual disability-cardiac arrhythmia syndrome
MONDO:0030036 leukoencephalopathy, motor delay, spasticity, and dysarthria syndrome
MONDO:0007631 chromosome 16p12.1 deletion syndrome, 520kb
MONDO:0014881 transketolase deficiency
MONDO:0008774 2-aminoadipic 2-oxoadipic aciduria
MONDO:0010452 intellectual disability, X-linked 90
MONDO:0013491 megalencephalic leukoencephalopathy w

In [117]:
# Phenotype to Gene
input_type = "biolink:PhenotypicFeature"
output_type = "biolink:Gene"
predicate = "biolink:genetically_associated_with"
input_is_subject = "True"

In [118]:
leigh_query = generate_trapi_query( input_type, output_type, leigh_phenotypes, predicate, input_is_subject)
response = requests.post( AC_URL, json = leigh_query )
print( response.status_code )

200


In [119]:
resp = response.json()
for result in resp["message"]["results"]:
    nb = result["node_bindings"]["output"][0]["id"]
    name = resp["message"]["knowledge_graph"]["nodes"][nb]["name"]
    print(nb, name)

NCBIGene:93986 FOXP2
NCBIGene:53335 BCL11A
NCBIGene:85358 SHANK3
NCBIGene:2077 ERF
NCBIGene:9378 NRXN1
NCBIGene:1806 DPYD
NCBIGene:26047 CNTNAP2
NCBIGene:26040 SETBP1
NCBIGene:284058 KANSL1
NCBIGene:50484 RRM2B
NCBIGene:113189 CHST14
NCBIGene:2903 GRIN2A
NCBIGene:5728 PTEN
NCBIGene:3737 KCNA2
NCBIGene:3954 LETM1
NCBIGene:4204 MECP2
NCBIGene:18 ABAT


In [120]:
resp["message"]["knowledge_graph"]["edges"]

{'e_HP:0000739_member_of_uuid_1': {'subject': 'HP:0000739',
  'object': 'uuid:1',
  'predicate': 'biolink:member_of',
  'sources': [{'resource_id': 'infores:answercoalesce',
    'resource_role': 'primary_knowledge_source'}],
  'attributes': [{'attribute_type_id': 'biolink:knowledge_level',
    'value': '???',
    'attribute_source': 'infores:answercoalesce'},
   {'attribute_type_id': 'biolink:agent_type',
    'value': '???',
    'attribute_source': 'infores:answercoalesce'}]},
 'e_HP:0001288_member_of_uuid_1': {'subject': 'HP:0001288',
  'object': 'uuid:1',
  'predicate': 'biolink:member_of',
  'sources': [{'resource_id': 'infores:answercoalesce',
    'resource_role': 'primary_knowledge_source'}],
  'attributes': [{'attribute_type_id': 'biolink:knowledge_level',
    'value': '???',
    'attribute_source': 'infores:answercoalesce'},
   {'attribute_type_id': 'biolink:agent_type',
    'value': '???',
    'attribute_source': 'infores:answercoalesce'}]},
 'e_HP:0001252_member_of_uuid_1': {'